# Identification of temporal knowledge

This notebook processes text from music personalities' biographies and extract historical meetups information 
 Pre-requirements:
    Text organised in sentences

The implementation of the algorithm is based in the work presented by Zhong et al.

    @inproceedings{Zhong_Sun_Cambria_2017, address={Vancouver, Canada}, 
    title={Time Expression Analysis and Recognition Using Syntactic Token Types and General Heuristic Rules}, 
    url={http://aclweb.org/anthology/P17-1039}, DOI={10.18653/v1/P17-1039}, booktitle={Proceedings of the 55th Annual Meeting of the Association for Computational Linguistics (Volume 1: Long Papers)}, publisher={Association for Computational Linguistics}, 
    author={Zhong, Xiaoshi and Sun, Aixin and Cambria, Erik}, year={2017}, pages={420–429}, language={en} }

The authors use HEURISTIC rules to identify time tokens, and POST tags to filter out ambiguos time tokens
Implementation in JAVA https://github.com/zhongxiaoshi/syntime

Implementation of the approach, from Zhong et al. analysis:
    1) identify time tokents, 2) identify time segments, 3) identify time expressions
    
Our implementation:

    - Identify time tokens, we use three types of tokens: TIME, MODIFIER, NUMERAL.
        Each type have more specific types:
            MODIFIER = ["PREFIX","SUFFIX","LINKAGE","COMMA","PARENTHESIS","INARTICLE"]
            NUMERAL = ["BASIC","DIGIT","ORDINAL"]
            TIME = ["DECADE", "YEAR", "SEASON", "MONTH", "WEEK", "DATE", "TIME", "DAY_TIME", "TIMELINE", "HOLIDAY", "PERIOD", "DURATION", "TIME_UNIT","TIME_ZONE", "ERA","MID","TIME_ZONE","DAY","HALFDAY"]
        Added PARENTHESIS and improving regular expressions
    - Initialize regular expressions:
        Read regular expressions stored in:
        - timeRegex.txt
    - Build additional regular expressions using the base expressions in timeRegex.txt
    - Compile regex objects just once for better performance
    
For each sentence in the biography:

    a) Identify token types. Function "def get_time_tokens(text):"
        - Tokenize 
        - Obtain POS tags 
        - Use regular expressions to identify type of token: time, modifier, numeral
        - Filter out ambiguous words by matching POS tags and type of token 
        - Output: A list of all the tokenized words, POS tags, type of token and token

    b) Identify time segments. Function "def get_time_segments(time_token_list):"
        - A time segment has one time token and one or zero modifiers or numerals
        - Search for a time token, once found search the surroundings:
        - Search tokens on the left 
          If PREFIX or NUMERAL or IN_ARTICLE continue searching
          - Search tokens on the right 
          If SUFIX or NUMERAL continue searching
             For right and left search, if token is COMMA or LINKAGE then stop
          If time segments overlap, then apply heuristic rules and merge segments
        - Output: A list of time segments, each time segment has the word's index in the sentence

    Our implementation
    c) Identify time expressions. Function "classify_type_time_expression(time_expression_list):"
        - Three types of time expressions: time point, time reference and time range
        - Apply heuristic rules to classify the type of time expression
        - Output: A dataframe with the sentence, the type of time expression, the time expression and indexes
        - Store each biography as a CSV file in extractedTimeExpressions/

Directories information:

    - indexedSentences/ : collection of biographies in CSV format. Each row of the file represents a sentence. Each row has a section name and paragraph index, and sentence index
    - extractedTimeExpressions/ : collection of annotated time expressions grouped by biography

In [1]:
# For nltk time entities
# time entity
import nltk.tokenize as nt
import nltk
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk import Tree
from nltk.stem import WordNetLemmatizer
# nltk.download('wordnet')

# if not installed, run the following lines
# nltk.download('averaged_perceptron_tagger')
# nltk.download('maxent_ne_chunker')
# nltk.download('words')
# nltk.download('punkt')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [32]:
import sys
import os
import pandas as pd
import re
import numpy as np

In [ ]:
# reading every CSV with indexed sentences
# return a list object of files in the given folder
files_list = [f for f in os.listdir('indexedSentences') if not f.startswith('.')]
# parse to dataframe
df_files = pd.DataFrame(files_list, columns=['file_name'])
print(len(df_files))

# df_files = df_files.query("file_name=='10002116.csv'")
# extract only the ones that do not exist in folder
files_list_index = [f for f in os.listdir('extractedTimeExpressions') if not f.startswith('.')]
# parse to dataframe
df_te = pd.DataFrame(files_list_index, columns=['file_name'])
print(len(df_te))
# Join with df_query to find the ones that are not processed yet
df_result = df_files[~df_files['file_name'].isin(df_te['file_name'])]
df_result.sort_values(by=['file_name'],ascending=True,inplace=True)
print(len(df_result))
df_result.to_csv('totalBiographiesTimeExpression.csv',index=False)
df_result.info()
df_result.head()

33309
33297
12


/opt/conda/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 12 entries, 2439 to 29469
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_name  12 non-null     object
dtypes: object(1)
memory usage: 192.0+ bytes


,file_name
2439,1328892.csv
3799,1511606.csv
5988,17506108.csv
8874,2177780.csv
10283,23822153.csv


In [3]:
# extract only the ones that do not exist in folder
files_list = [f for f in os.listdir('extractedEntities') if not f.startswith('.')]
# parse to dataframe
df_query = pd.DataFrame(files_list, columns=['file_name'])
df_result = df_files[~df_files['file_name'].isin(df_query['file_name'])]
df_files = df_result
df_files.info()
df_files.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_name  0 non-null      object
dtypes: object(1)
memory usage: 0.0+ bytes


,file_name


# HEURISTIC RULES

In [3]:
def read_text_file(filename,folder):
    fileObject = open(folder+filename, "r")
    data = fileObject.readlines()
    return data

def parse_text_to_dict(textFile):
    regexDictionary = {}
    for line in textFile:
        # print(line)
        if len(line.strip())>1:
            item = line.strip().split("\t")
            # print(item)
            regexDictionary[item[0][1:len(item[0])-1]] = item[1]
    return regexDictionary

# Source: SynTimeRegex.java -> inducePattern()
def get_patterns(regex_dict):
    new_patterns = {}
    # POS: CD; Type: DATE
    new_patterns["YEAR_MID_REGEX"] = "(" + regex_dict["MID_REGEX"] + ")(" + regex_dict["YEAR_REGEX_1"] + "|" + regex_dict["YEAR_REGEX_2"] + ")"
    new_patterns["YEAR_YEAR_REGEX"] = regex_dict["YEAR_REGEX_1"] + "-(" + regex_dict["YEAR_REGEX_1"] + "|[0-9]{2})"
    # print(new_patterns)
    
    # POS: NNP; Type: DATE*/
    new_patterns["MONTH_MID_REGEX"] = "(" + regex_dict["MID_REGEX"] + ")(" + regex_dict["MONTH_REGEX"] + "|" + regex_dict["MONTH_ABBR_REGEX"] + ")"
    new_patterns["MONTH_MONTH_REGEX"] = "(" + regex_dict["MONTH_REGEX"] + "|" + regex_dict["MONTH_ABBR_REGEX"] + ")-(" + regex_dict["MONTH_REGEX"] + "|" + regex_dict["MONTH_ABBR_REGEX"] + ")"
    new_patterns["YEAR_MONTH_REGEX_1"] = regex_dict["YEAR_REGEX_1"] + "-(" + regex_dict["MONTH_REGEX"] + "|" + regex_dict["MONTH_ABBR_REGEX"] + ")"
    new_patterns["YEAR_MONTH_REGEX_2"] = "(" + regex_dict["MONTH_REGEX"] + "|" + regex_dict["MONTH_ABBR_REGEX"] + ")-" + regex_dict["YEAR_REGEX_1"]
    
    # POS: NNP; Type: DATE*/
    new_patterns["WEEK_WEEK_REGEX"] = "(" + regex_dict["WEEK_REGEX"] + "|" + regex_dict["WEEK_ABBR_REGEX"] + ")-(" + regex_dict["WEEK_REGEX"] + "|" + regex_dict["WEEK_ABBR_REGEX"] + ")"

    # POS: CD*/
    new_patterns["BASIC_NUMBER_NUMBER_REGEX"] = "(" + regex_dict["BASIC_NUMBER_REGEX_1"] + ")-(" + regex_dict["BASIC_NUMBER_REGEX_1"] + ")"

    # POS: CD*/
    new_patterns["DIGIT_REGEX_2"] = regex_dict["DIGIT_REGEX_1"] + "[/\\.]" + regex_dict["DIGIT_REGEX_1"]
    new_patterns["DIGIT_DIGIT_REGEX"] = regex_dict["DIGIT_REGEX_1"] + "-" + regex_dict["DIGIT_REGEX_1"]

    # POS: JJ, CD */
    new_patterns["ORDINAL_ORDINAL_REGEX"] = "(" + regex_dict["ORDINAL_REGEX_1"] + ")-(" + regex_dict["ORDINAL_REGEX_1"] + ")"

    # Type: TIME*/
    new_patterns["TIME_TIME_REGEX"] = "(" + regex_dict["TIME_REGEX_1"] + "|" + regex_dict["TIME_REGEX_2"] + ")-(" + regex_dict["TIME_REGEX_1"] + "|" + regex_dict["TIME_REGEX_2"] + ")"
    new_patterns["ERA_YEAR_REGEX"] = regex_dict["DIGIT_REGEX_1"] + "(" + regex_dict["ERA_REGEX"] + ")"

    # POS: NN, VBD; Type: TIME */
    new_patterns["HALFDAY_REGEX_2"] = "(" + regex_dict["DIGIT_REGEX_1"] + "|" + new_patterns["DIGIT_REGEX_2"] + "|" + regex_dict["TIME_REGEX_1"] + "|" + regex_dict["TIME_REGEX_2"] + ")(" + regex_dict["HALFDAY_REGEX_1"] + ")"
    new_patterns["HALFDAY_HALFDAY_REGEX"] = new_patterns["HALFDAY_REGEX_2"] + "-" + new_patterns["HALFDAY_REGEX_2"]

    # POS: NNS; Type: DATE*/
    new_patterns["DECADE_MID_REGEX"] = "(" + regex_dict["MID_REGEX"] + ")(" + regex_dict["DECADE_REGEX"] + ")"

    # POS: NN, JJ; Type: DURATION*/
    new_patterns["DURATION_REGEX"] = "(" + regex_dict["DIGIT_REGEX_1"] + "|" + new_patterns["DIGIT_REGEX_2"] + "|" + regex_dict["BASIC_NUMBER_REGEX_1"] + "|" + regex_dict["BASIC_NUMBER_REGEX_2"] + "|" + regex_dict["ORDINAL_REGEX_1"] + "|" + regex_dict["ORDINAL_REGEX_2"] + "|" + regex_dict["INARTICLE_REGEX"] + ")-?(" + regex_dict["TIME_UNIT_REGEX"] + ")"

    new_patterns["DURATION_DURATION_REGEX_1"] = "(" + new_patterns["DIGIT_DIGIT_REGEX"] + "|" + new_patterns["BASIC_NUMBER_NUMBER_REGEX"] + "|" + new_patterns["ORDINAL_ORDINAL_REGEX"] + ")-?(" + regex_dict["TIME_UNIT_REGEX"] +")"
    new_patterns["DURATION_DURATION_REGEX_2"] = new_patterns["DURATION_REGEX"] + "-" + new_patterns["DURATION_REGEX"]

    # POS: NN, NNP, NNS, RB, JJ; Type: TIME*/
    new_patterns["DAY_TIME_MID_REGEX"] = "(" + regex_dict["MID_REGEX"] + ")(" + regex_dict["DAY_TIME_REGEX"] + ")"

    # POS: NN, NNS; Type: DATE*/
    new_patterns["SEASON_MID_REGEX"] = "(" + regex_dict["MID_REGEX"] + ")(" + regex_dict["SEASON_REGEX"] + ")"
    
    return new_patterns

# returns compiled regex objects
def get_compiled_regex(patterns_dict):
    compiled_regex_dict = {}
    for regex in patterns_dict:
        if regex == "TIME_ZONE_REGEX":
            compiled_regex_dict[regex] = re.compile(patterns_dict[regex])
        else:
            compiled_regex_dict[regex] = re.compile(patterns_dict[regex],flags=re.IGNORECASE)
    return compiled_regex_dict

def get_token_type(regex_name):
    regex_name = regex_name[:regex_name.find("_")]
    types_dict = {}
    types_dict["MODIFIER"]=["PREFIX","SUFFIX","LINKAGE","COMMA","PARENTHESIS","INARTICLE"]
    types_dict["NUMERAL"]=["BASIC","DIGIT","ORDINAL"]
    types_dict["TIME"] = ["DECADE", "YEAR", "SEASON", "MONTH", "WEEK", "DATE", "TIME", "DAY_TIME", "TIMELINE",
                      "HOLIDAY", "PERIOD", "DURATION", "TIME_UNIT","TIME_ZONE", "ERA","MID","TIME_ZONE","DAY","HALFDAY"]
    for type_name in types_dict:
        if regex_name in types_dict[type_name]:
            return type_name


In [4]:
# TIME TOKENS RULES
# 15 types of tokens: 
#  DECADE, YEAR, SEASON, MONTH, WEEK, DATE, TIME, DAY_TIME, TIMELINE, 
#  HOLIDAY, PERIOD, DURATION, TIME_UNIT,TIME_ZONE, ERA
# MODIFIERS TOKEN
#  5 TYPES
#  PREFIX, SUFFIX, LINKAGE, COMMA, IN_ARTICLE
#  ** adding PARENTHESIS
# NUMERAL TOKENS

# 1.1 Read regex
textFile = read_text_file("","timeRegex.txt")
# print(textFile)
regex_dict = parse_text_to_dict(textFile)
# print(len(regex_dict))
new_patterns_dict = get_patterns(regex_dict)
# print(len(new_patterns_dict))
regex_dict.update(new_patterns_dict)
# print(len(new_patterns_dict))
compiled_regex_dict = get_compiled_regex(regex_dict)

/tmp/ipykernel_665/825276394.py:75: DeprecationWarning: Flags not at the start of the expression '((?i)[2-9]?1st|[2-9]' (truncated)
  compiled_regex_dict[regex] = re.compile(patterns_dict[regex],flags=re.IGNORECASE)
/tmp/ipykernel_665/825276394.py:75: DeprecationWarning: Flags not at the start of the expression '(\\d+|\\d+[/\\.]\\d+|one' (truncated)
  compiled_regex_dict[regex] = re.compile(patterns_dict[regex],flags=re.IGNORECASE)
/tmp/ipykernel_665/825276394.py:75: DeprecationWarning: Flags not at the start of the expression '(\\d+-\\d+|(one|two|th' (truncated)
  compiled_regex_dict[regex] = re.compile(patterns_dict[regex],flags=re.IGNORECASE)


In [5]:
def print_tokens_list(tokens_list):
    for item in tokens_list:
        print(item)
        
# tokenize POS tags and identify token type
# source: InduceTokenType.java -> getTokenType()
def get_time_tokens(text):
    tokenized_sent=nt.word_tokenize(text)
    pos_sentences=nltk.pos_tag(tokenized_sent)
    # print(pos_sentences)

    tokens_list = []
    index = 0
    # for each word in the text
    for token in pos_sentences:
        # match regex, call objects already compiled
        for compile_pattern in compiled_regex_dict:

            # print(type(compiled_regex_dict))
            # order in list [word - 0, POS - 1, time_token - 2, type_token - 3, index - 4]
            result = compiled_regex_dict[compile_pattern].match(token[0])
            if result != None:
                ## use POS tags to identify time expressions and avoid things like MAY (a month) from may (I may leave)
                # use a flag to assign values
                valid = False
                # YEAR_REGEX_1, YEAR_REGEX_2, YEAR_MID, ERA_YEAR
                if compile_pattern.startswith("YEAR_REGEX") or compile_pattern.startswith("YEAR_MID") or compile_pattern.startswith("ERA_YEAR"):
                    valid = True
                
                # YEAR_YEAR
                if compile_pattern.startswith("YEAR_YEAR"):
                    valid = True
                # SEASON_REGEX, SEASON_MID
                if compile_pattern.startswith("SEASON_REGEX") and token[1].startswith("NN") or compile_pattern.startswith("SEASON_MID"):
                    valid = True
                # MONTH_REGEX, MONTH_MID
                if compile_pattern.startswith("MONTH_REGEX") and token[1].startswith("NN") or compile_pattern.startswith("MONTH_MID"):
                    valid = True
                # MONTH_ABBR, JJ?
                if compile_pattern.startswith("MONTH_ABBR") and (token[1].startswith("NN") or token[1].startswith("JJ")):
                    valid = True
                # MONTH_MONTH
                if compile_pattern.startswith("MONTH_MONTH"):
                    valid = True
                # YEAR_MONTH_1, YEAR_MONTH_2
                if compile_pattern.startswith("YEAR_MONTH"):
                    valid = True
                # WEEK_REGEX, 
                if compile_pattern.startswith("WEEK_REGEX") and token[1].startswith("NN"):
                    valid = True
                # WEEK_ABBR
                if compile_pattern.startswith("WEEK_ABBR") and token[1].startswith("NN"):
                    valid = True
                # WEEK_WEEK
                if compile_pattern.startswith("WEEK_WEEK"):
                    valid = True
                # DATE_REGEX_1, 2, 3
                if compile_pattern.startswith("DATE_REGEX"):
                    valid = True
                # TIME_REGEX_1, 2
                if compile_pattern.startswith("TIME_REGEX"):
                    valid = True
                # TIME_TIME
                if compile_pattern.startswith("TIME_TIME"):
                    valid = True
                # HALFDAY_REGEX_1, 2
                if compile_pattern.startswith("HALFDAY_REGEX"):
                    valid = True
                # HALFDAY_HALFDAY
                if compile_pattern.startswith("HALFDAY_HALFDAY"):
                    valid = True
                # TIME_ZONE
                if compile_pattern.startswith("TIME_ZONE"):
                    valid = True
                # ERA_REGEX
                if compile_pattern.startswith("ERA_REGEX") and token[1].startswith("NN"):
                    valid = True
                # TIME_UNIT
                if compile_pattern.startswith("TIME_UNIT") and token[1].startswith("NN"):
                    valid = True
                # DURATION_REGEX
                if compile_pattern.startswith("DURATION_REGEX") and (token[1].startswith("NN") or token[1] == "JJ" or token[1] == "CD"):
                    valid = True
                # DURATION_DURATION_1, 2
                # if compile_pattern.startswith("DURATION_DURATION"):
                if not compile_pattern.startswith("DURATION_REGEX") and (compile_pattern.startswith("DURATION_DURATION_REGEX_1") or compile_pattern.startswith("DURATION_DURATION_REGEX_2")):
                    valid = True
                # DAY_TIME_REGEX, DAY_TIME_MID
                if compile_pattern.startswith("DAY_TIME_REGEX") and (token[1].startswith("NN") or token[1] == "RB" or token[1] == "JJ") or compile_pattern.startswith("DAY_TIME_MID") :
                    valid = True
                # TIMELINE_REGEX
                if compile_pattern.startswith("TIMELINE_REGEX") and (token[1].startswith("NN") or token[1] == "RB"):
                    valid = True
                # HOLIDAY
                if compile_pattern.startswith("HOLIDAY"):
                    valid = True
                # PERIOD_REGEX
                if compile_pattern.startswith("PERIOD_REGEX") and (token[1].startswith("NN") or token[1] == "RB" or token[1] == "JJ"):
                    valid = True
                # DECADE_REGEX, DECADE_MID
                if compile_pattern.startswith("DECADE_REGEX") and token[1].startswith("NN") or compile_pattern.startswith("DECADE_MID"):
                    valid = True
                # DIGIT_1,2, BASIC_NUMBER_REGEX_1,2, ORDINAL_REGEX_1,2
                if compile_pattern.startswith("DIGIT_REGEX") or compile_pattern.startswith("BASIC_NUMBER_R") or ((compile_pattern.startswith("ORDINAL_REGEX_1") or compile_pattern.startswith("ORDINAL_REGEX_2"))  and (token[1] == "JJ" or token[1] == "CD" or token[1] == "RB")):
                    valid = True
                    
                # DIGIT_DIGIT, BASIC_NUMBER_NUMBER, ORDINAL_ORDINAL
                # if compile_pattern.startswith("DIGIT_DIGIT") or compile_pattern.startswith("BASIC_NUMBER_NUMBER") or compile_pattern.startswith("ORDINAL_ORDINAL"):
                if not compile_pattern.startswith("YEAR_YEAR") and not compile_pattern.startswith("BASIC_NUMBER_REGEX_2") and (compile_pattern.startswith("DIGIT_DIGIT") or compile_pattern.startswith("BASIC_NUMBER_NUMBER") or compile_pattern.startswith("ORDINAL_ORDINAL")):
                    valid = True
                # INARTICLE_REGEX
                if compile_pattern.startswith("INARTICLE_REGEX"):
                    valid = True
                # PREFIX_REGEX_1,2
                if compile_pattern.startswith("PREFIX_REGEX_1") and not token[1].startswith("NN") or compile_pattern.startswith("PREFIX_REGEX_2") and token[1].startswith("NN"):
                    valid = True
                # SUFFIX
                if compile_pattern.startswith("SUFFIX"):
                    valid = True
                # LINKAGE
                if compile_pattern.startswith("LINKAGE"):
                    valid = True
                # LINKAGE
                if compile_pattern.startswith("COMMA"):
                    valid = True
                    
                if valid:
                    item = [token[0],token[1],compile_pattern,get_token_type(compile_pattern),index]
                else:
                    item = [token[0],token[1],"","",index]
                break
            else:
                item = [token[0],token[1],"","",index]
                
        tokens_list.append(item)
        index += 1

    # print_tokens_list(tokens_list)
    return tokens_list

# extract time expressions using time tokens
def extract_time_expression(index_list,time_token_list):
    expressions = []
    for row in index_list:
        # print(row)
        expression = ""
        for i in range(min(row), max(row)+1):
            # print(i)
            expression = expression + time_token_list[i][0] + " "
            
        expressions.append(expression.rstrip())
    return expressions
    # print(expressions)
    
# extract time expressions using time tokens
def extract_time_expression_extended(index_list,time_token_list):
    # order in list [word - 0, POS - 1, time_token - 2, type_token - 3, index - 4]
    expressions = []
    for row in index_list:
        # print(row)
        expression = ""
        POS_tag = ""
        time_token = ""
        type_token = ""
        for i in range(min(row), max(row)+1):
            # print(i)
            expression = expression + time_token_list[i][0] + " "
            POS_tag = POS_tag + time_token_list[i][1] + " "
            time_token = time_token + time_token_list[i][2] + " "
            type_token = type_token + time_token_list[i][3] + " "
            
        expressions.append([expression.rstrip(),POS_tag.rstrip(),time_token.rstrip(),type_token.rstrip()])
    # print(expressions)
    return expressions
    
# identify time segments
# Source: SynTime.java-> identifyTimeSegment, identifyTimeToken
def get_time_segments(time_token_list):
    # order in list [word - 0, POS - 1, time_token - 2, type_token - 3, index - 4]
    # order in list
    time_segment = []
    index_token = 0
    # for token in time_token_list:
    while index_token < len(time_token_list):
        # print(index_token)
        current_token = time_token_list[index_token]
        type_token = current_token[3]
        # search time tokens
        if type_token == "TIME":
            type_expression = current_token[2]
            if type_expression.startswith("DURATION") or type_expression.startswith("PERIOD"):
                # print(current_token[4])
                segment = []
                segment.append(index_token)
                time_segment.append(segment)
                index_token +=1
                continue
            # # insert here am time and pm validation
            if type_expression.startswith("HALFDAY") and not index_token == 0 and (current_token[0] == "am" or current_token[0] == "pm"):
                # print(time_token_list[index_token-1][3])# ['am', 'VBP', 'HALFDAY_REGEX_1', 'TIME', 1]
                if not (time_token_list[index_token-1][3].startswith("NUMERAL") or time_token_list[index_token-1][3].startswith("TIME")):
                    # print("continue") 
                    index_token +=1
                    continue
            # if TIMELINE_REGEX, cases when 'present' is not correctly formed
            if type_expression.startswith("TIMELINE") and current_token[0] == "present":
                if index_token != 0 and not (time_token_list[index_token-1][3].startswith("NUMERAL") or time_token_list[index_token-1][3].startswith("TIME")):
                    # segment = []
                    index_token +=1
                    continue
                if ((index_token+1)<len(time_token_list)) and not (time_token_list[index_token+1][3].startswith("NUMERAL") or time_token_list[index_token+1][3].startswith("TIME")):
                    index_token +=1
                    continue
            # else:
            # start searching time segments
            segment = []
            # current token
            index = current_token[4]
            # time_token_one = True
            segment.append(index)
            # print(current_token)

            # search tokens on the left
            # reverse loop (current index -1 to 0)
            for i in reversed(range(index)):
                # previous token
                prev_token = time_token_list[i]
                # print("prev token left",prev_token)
                if prev_token[2].startswith("PREFIX") or prev_token[3].startswith("NUMERAL") or prev_token[2].startswith("INARTICLE"):
                    segment.append(i)
                elif prev_token[2].startswith("COMMA"):
                    break
                elif prev_token[2].startswith("LINKAGE"): #and time_token_list[i-1][3].startswith("NUMERAL"):
                    if type_token.startswith("TIME") and time_token_list[i-1][3].startswith("NUMERAL"):
                        segment.append(i)
                    else:
                        break
                    # segment.append(i)
                else:
                    break
                # if prev_token[3] == "":
                #     break
                # elif prev_token[2].startswith("COMMA") or prev_token[2].startswith("LINKAGE") or prev_token[2].startswith("PARENTHESIS"):
                #     break
                # elif prev_token[2].startswith("PREFIX") or prev_token[3].startswith("NUMERAL") or prev_token[2].startswith("INARTICLE"):
                #     segment.append(i)
                # # merge two time segments
                # elif prev_token[3].startswith("TIME") and len(segment)==1:
                #     segment.append(i)
                # elif prev_token[3].startswith("TIME") and len(segment)>1:
                #     if not time_token_list[min(segment)][2].startswith("COMMA") or not time_token_list[min(segment)][2].startswith("LINKAGE"):
                #         segment.append(i)

            # search right
            for i in range(index+1, len(time_token_list)):
                next_token = time_token_list[i]
                # print("sub token right")
                # print(next_token)
                if next_token[2].startswith("SUFFIX") or next_token[3].startswith("NUMERAL"):
                    segment.append(i)
                elif next_token[2].startswith("COMMA"):
                    if ((i+1)<len(time_token_list)) and time_token_list[i+1][3].startswith("TIME"):
                        segment.append(i)
                    else:
                        break
                elif next_token[2].startswith("LINKAGE") and ((i+1)<len(time_token_list)) and time_token_list[i+1][3].startswith("TIME"):
                    segment.append(i)
                elif next_token[2].startswith("LINKAGE"): 
                    if type_token.startswith("TIME") and ((i+1)<len(time_token_list)) and time_token_list[i+1][3].startswith("NUMERAL"):
                        segment.append(i)
                    else:
                        break
                elif next_token[3].startswith("TIME") and len(segment)>=1:
                    segment.append(i)
                elif next_token[3] == "":
                    break
                # elif next_token[2].startswith("COMMA") or next_token[2].startswith("LINKAGE") or next_token[2].startswith("PARENTHESIS") or next_token[2].startswith("INARTICLE"):
                #     break
                # elif next_token[2].startswith("SUFFIX") or next_token[3].startswith("NUMERAL"):
                #     segment.append(i)
                # # merge two time segments
                # elif next_token[3].startswith("TIME") and len(segment)==1:
                #     segment.append(i)
                # elif next_token[3].startswith("TIME") and len(segment)>1:
                #     if not time_token_list[max(segment)][2].startswith("COMMA") or not time_token_list[max(segment)][2].startswith("LINKAGE"):
                #         segment.append(i)
                else:
                    break
            
            # calculate next item in list
            index_token = max(segment)
            # print(index_token)
            # add to the list of all time segments in sentence
            time_segment.append(segment)
            # print(segment)
            # break
        index_token +=1
    return time_segment

def classify_type_time_expression(time_expression_list):
    # print(len(time_expression_list))
    type_time_expression_name = []
    for time_expression_details in time_expression_list:
        # input: a lis of list with the following information ['time_expression', 'POST_tag',"time_token","type_token"]
        POS_tag_list = time_expression_details[1].split()
        # print(POS_tag_list)
        POS_tag_lenght = len(POS_tag_list)

        if POS_tag_list[0] == "CD":
            # eg. 1924	CD
            if POS_tag_lenght == 1:
                type_time_expression_name.append("time_reference")
            # 1899 and 1920	CD CC CD
            # 1846 to 1885	CD TO CD
            elif POS_tag_lenght == 3 and (POS_tag_list[1] == "CC" or POS_tag_list[1] == "TO"):
                type_time_expression_name.append("time_range")
            # 30-Jun-02	CD NNP CD
            elif POS_tag_lenght == 3 and POS_tag_list[1] == "NNP":
                type_time_expression_name.append("time_point")
            # eg. 2 June 1857 - 23 February 1934	CD NNP CD NNP CD NNP CD
            elif POS_tag_lenght>3:
                type_time_expression_name.append("time_range")
            # three years	CD NNS
            elif POS_tag_lenght == 2 and (POS_tag_list[1] == "NNS" or POS_tag_list[1] == "RB"):
                type_time_expression_name.append("time_reference")
            # Eight years older	CD NNS JJR
            elif POS_tag_lenght == 3 and POS_tag_list[1] == "NNS" and POS_tag_list[2] == "JJR":
                type_time_expression_name.append("time_reference")
            else:
                type_time_expression_name.append("")
        elif POS_tag_list[0] == "DT":
            # # the 1890s	DT CD
            # if POS_tag_lenght == 2 and POS_tag_list[1] == "CD":
            #     return "time_reference"
            # # the early 1920s	DT JJ CD
            # # the next seven years	DT JJ CD NNS
            # if (POS_tag_lenght == 3 or POS_tag_lenght == 4 or POS_tag_lenght == 5) and POS_tag_list[1] == "JJ":
            #     return "time_reference"
            # # some years	DT NNS
            # # a decade	DT NN
            # # the beginning of the twentieth century	DT NN IN DT JJ NN
            # if POS_tag_list[1].startswith("NN"):
            #     return "time_reference"
            type_time_expression_name.append("time_reference")
        # of 1910	IN CD
        # of the 1890s	IN DT CD
        # of the first march	IN DT JJ NN
        elif POS_tag_list[0] == "IN":
            type_time_expression_name.append("time_reference")
        # early 2007	JJ CD
        # many years	JJ NNS
        elif POS_tag_list[0] == "JJ":
            type_time_expression_name.append("time_reference")
        # of 1910	IN CD
        # of the 1890s	IN DT CD
        # of the first march	IN DT JJ NN
        elif POS_tag_list[0] == "JJR":
            type_time_expression_name.append("time_reference")
        # Sep-05	NNP CD
        # forties	NNS
        elif POS_tag_list[0].startswith("NN"):
            type_time_expression_name.append("time_reference")
        # now	RB
        # nearly ten years	RB JJ NNS
        # more than three years	RBR IN CD NNS
        elif POS_tag_list[0].startswith("RB"):
            type_time_expression_name.append("time_reference")
        # off the next year	RP DT JJ NN
        elif POS_tag_list[0].startswith("RP"):
            type_time_expression_name.append("time_reference")
        # lasting less than fifteen minutes	VBG JJR IN JJ NNS
        elif POS_tag_list[0].startswith("VBG"):
            type_time_expression_name.append("time_reference")
        else:
            type_time_expression_name.append("")

    # print(len(type_time_expression_name))
    return type_time_expression_name

In [8]:
# for file_name_item in df_files.itertuples():
for chunk in pd.read_csv('list_wikiIdSample.csv', chunksize=50):
# for chunk in pd.read_csv('totalBiographiesTimeExpression.csv', chunksize=50):
# for chunk in pd.read_csv('totalBiographiesBenchmark.csv', chunksize=50):
    df_file_name = pd.DataFrame()
    df_file_name['file_name'] = chunk['file_name']
    for file_name_item in df_file_name.itertuples():
        # if file with time expressions extraction already exist, skip to next one
        if os.path.isfile('extractedTimeExpressions/'+file_name_item.file_name.replace(".txt",".csv")):
            print("Extraction done. ", file_name_item.file_name)
        else:
            file_exists = os.path.isfile('indexedSentences/'+file_name_item.file_name.replace(".txt",".csv"))
            
            if file_exists:
                print(file_name_item.file_name.replace(".txt",".csv"))
                # read the cached results from the query
                biography_df = pd.read_csv('indexedSentences/'+file_name_item.file_name.replace(".txt",".csv"))
                df_time_expression = pd.DataFrame()

                # for each sentence in the biography
                for sentence_row in biography_df.itertuples():
                    # print(sentence_row.sentences)
                    # validate if the sentence is NaN
                    if pd.isna(sentence_row.sentences):
                        continue
                    # identify time tokens
                    time_token_list = get_time_tokens(sentence_row.sentences)
                    # print(time_token_list)
                    # identify time_segments
                    time_segments = get_time_segments(time_token_list)

                    if len(time_segments) > 0:
                        # print(time_segments)
                        time_expressions_list = extract_time_expression_extended(time_segments,time_token_list)

                        df_temp = pd.DataFrame(time_expressions_list, columns=['time_expression', 'POST_tag',"time_token","type_token"])
                        df_temp["time_expression_type"] = classify_type_time_expression(time_expressions_list)
                        df_temp["sentences"] = sentence_row.sentences
                        df_temp["sentenceIndex"] = sentence_row.sentenceIndex
                        df_temp["paragraphIndex"] = sentence_row.paragraphIndex
                        df_temp["section"] = sentence_row.section
                        df_temp["wikiId"] = sentence_row.wikiId
                        df_time_expression = df_time_expression.append(df_temp)
                if not df_time_expression.empty:
                    df_time_expression.drop(['section','wikiId','sentences'], axis=1,inplace=True)
                    # print("total.",sys.getsizeof(df_time_expression))
                    df_time_expression.to_csv('extractedTimeExpressions/'+file_name_item.file_name.replace(".txt",".csv"),index=False)
                else:
                    print("No time expressions extracted.", file_name_item.file_name)
            else:
                print("File does not exist. ", file_name_item.file_name)

Extraction done.  10085.csv
Extraction done.  9039.csv
Extraction done.  21511.csv
Extraction done.  45181.csv
Extraction done.  49644.csv
Extraction done.  50350.csv
Extraction done.  57520.csv
Extraction done.  99636.csv
Extraction done.  180714.csv
Extraction done.  312443.csv
Extraction done.  2444917.csv
Extraction done.  608845.csv
Extraction done.  1048151.csv
Extraction done.  2232977.csv
Extraction done.  1913885.csv
Extraction done.  1790990.csv
Extraction done.  409969.csv
Extraction done.  1709886.csv
Extraction done.  1551347.csv
Extraction done.  50782750.csv
Extraction done.  181946.csv
Extraction done.  579599.csv
Extraction done.  50963136.csv
Extraction done.  226142.csv
Extraction done.  1422240.csv
Extraction done.  1174545.csv
Extraction done.  70020.csv
Extraction done.  752694.csv
Extraction done.  312781.csv
Extraction done.  2898019.csv
Extraction done.  2253021.csv
Extraction done.  3450382.csv
Extraction done.  2553865.csv
Extraction done.  3263983.csv
Extrac

## NORMALISATION

### Process normalisation for dates:
    Approach in steps:
        We identified three cases for normalisation: time_point, time_range and time_reference
        1. Time point
            Time point is usually an specific date, typically represented in the format YYYY-MM-DD.
            With the aim to facilitate the representation using RDF and later publication of the KG, we calculate a beginning and end date, that are always the same
            We use the libraries datetil to format such cases. The object
            parser.parse("text").strftime("format")
        2. Time reference
            Time reference (including incomplete dates)
            Formatting using dateutil and approx_dates libraries. 
            Beginning and end dates calculated using the approx library. For a year, start is JAN and end is DIC, for YYYY-MM beginning is month's first day and end is month's last day
        3. Time ranges

In [33]:
from dateutil import parser
from  dateutil.parser import parse
from dateutil.relativedelta import *
from approx_dates.models import ApproxDate

In [2]:
# !pip install approx_dates

Using legacy 'setup.py install' for approx-dates, since package 'wheel' is not installed.
    Running setup.py install for approx-dates ... done
You should consider upgrading via the '/data/user-data/amt557/meetups_env/bin/python3.8 -m pip install --upgrade pip' command.


In [5]:
# Current list of biographies in MEETUPS
bioListMEETUPS_df = pd.read_csv('list_wikiIdSample.csv')
# bioListMEETUPS_df.rename(columns={'id': 'file_name','s':'resource'}, inplace=True)
# bioListMEETUPS_df['file_name'] = bioListMEETUPS_df['file_name'].astype(str) + '.csv'
bioListMEETUPS_df.drop_duplicates(subset=['file_name'],inplace=True)
bioListMEETUPS_df.info()
files_list_index = [f for f in os.listdir('extractedTimeExpressions/') if not f.startswith('.')]
df_te = pd.DataFrame(files_list_index, columns=['file_name'])
df_te.drop_duplicates(subset=['file_name'],inplace=True)
print("Total biographies: ",len(df_te))
df_result = df_te[~df_te['file_name'].isin(bioListMEETUPS_df['file_name'])]
print("Left to query: ",len(df_result))
df_result.to_csv('totalBiosTimeExpression.csv',index=False)
df_result.info()
df_result.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1011 entries, 0 to 1010
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_name  1011 non-null   object
dtypes: object(1)
memory usage: 15.8+ KB
Total biographies:  33863
Left to query:  32853
<class 'pandas.core.frame.DataFrame'>
Int64Index: 32853 entries, 0 to 33862
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_name  32853 non-null  object
dtypes: object(1)
memory usage: 513.3+ KB


,file_name
0,22107033.csv
1,9267481.csv
2,62240418.csv
3,7561075.csv
4,68262184.csv


In [3]:
# Reading the biographies in chunks of 50
# for chunk in pd.read_csv('../totalBiographiesBenchmark.csv', chunksize=50):
# for chunk in pd.read_csv('list_wikiIdSample.csv', chunksize=50):
for chunk in pd.read_csv('totalBiosTimeExpression.csv', chunksize=50):
    df_file_name = pd.DataFrame()
    df_file_name['file_name'] = chunk['file_name']
    # for each file
    for file_name_item in df_file_name.itertuples():
        file_exists = os.path.isfile('extractedTimeExpressions/'+file_name_item.file_name.replace(".txt",".csv"))
        
        if file_exists:
            print(file_name_item.file_name.replace(".txt",".csv"))
            # read the biography organised by sentences
            # try when the file is empty, no time expressions found
            try:
                biography_df = pd.read_csv('extractedTimeExpressions/'+file_name_item.file_name.replace(".txt",".csv"))
            except pd.errors.EmptyDataError:    
                print("EmptyDataError: No columns to parse from file")
                continue
            # print(biography_df.groupby(['time_expression_type'])['time_expression_type'].count())
            # for each sentence in the biography, identify if they can be a 
            # meetupsEval_df = evaluateCandidateMeetupsEntities(biography_df)
            # print(len(biography_df))
            # for each sentence in the biography
            list_time_nrml = []
            # time_expression=[]
            for sentence_row in biography_df.itertuples():
            # store results
                try:
                    startDate = ""
                    endDate = ""
                    nrmlsDate = ""
                    # print(sentence_row.time_expression_type,sentence_row.time_expression)
                    if sentence_row.time_expression_type == "time_point":
                        nrmlsDate = parser.parse(sentence_row.time_expression).strftime("%Y-%m-%d")
                        startDate = nrmlsDate
                        endDate = nrmlsDate
                        
                    if sentence_row.time_expression_type == "time_reference":
                        if "COMMA_REGEX" in sentence_row.time_token:
                            # special case for different format: November 10 , 1888
                            nrmlsDate = parser.parse(sentence_row.time_expression).strftime("%Y-%m-%d")
                            startDate = nrmlsDate
                            endDate = nrmlsDate
                        # special case for month year expressions
                        if sentence_row.time_token == "MONTH_REGEX YEAR_REGEX_1":
                            nrmlsDate = ApproxDate.from_iso8601(parser.parse(sentence_row.time_expression).strftime("%Y-%m"))
                            startDate = nrmlsDate.earliest_date
                            endDate = nrmlsDate.latest_date  
                        elif sentence_row.time_token == "PREFIX_REGEX_1 YEAR_REGEX_1" or sentence_row.time_token == "SEASON_REGEX YEAR_REGEX_1": #and (sentence_row.time_expression.split()[0].lower().startswith("the")):
                            # e.g the 1890s, this indicates a decade, therefore, endDate should be 9 years
                            if sentence_row.time_expression[-1] == 's' and (sentence_row.time_expression.split()[0].lower().startswith("the")):
                                expression = sentence_row.time_expression[:-1].split()[1]
                                nrmlsDate = ApproxDate.from_iso8601(parser.parse(expression).strftime("%Y"))
                                startDate = nrmlsDate.earliest_date
                                endDecade = parse(expression)+relativedelta(years=+9)
                                endDate = ApproxDate.from_iso8601(endDecade.strftime("%Y")).latest_date
                            else:
                                # E.g.1: Spring 1774 - SEASON_REGEX YEAR_REGEX_1
                                # E.g.2 sentence_row.time_expression.split()[0].lower().startswith("around"):
                                nrmlsDate = ApproxDate.from_iso8601(parser.parse(sentence_row.time_expression.split()[1]).strftime("%Y"))
                                startDate = nrmlsDate.earliest_date
                                endDate = nrmlsDate.latest_date
                        elif ("–") in str(sentence_row.time_expression):
                            startDate = ApproxDate.from_iso8601(parser.parse(sentence_row.time_expression.split("–")[0]).strftime("%Y")).earliest_date
                            endDate = ApproxDate.from_iso8601(parser.parse(sentence_row.time_expression.split("–")[1]).strftime("%Y")).latest_date
                        elif sentence_row.time_token == "MONTH_REGEX LINKAGE_REGEX MONTH_REGEX YEAR_REGEX_1":
                        # E.g.: November and December 1923 -> MONTH_REGEX LINKAGE_REGEX MONTH_REGEX YEAR_REGEX_1
                        # two months, range between two months in the same year
                            timeExpression_str = sentence_row.time_expression.replace("and","")
                            timeExpression_str = " ".join(timeExpression_str.split())
                            nrmlsDate = ApproxDate.from_iso8601(parser.parse(timeExpression_str.split()[0]+" "+timeExpression_str.split()[2]).strftime("%Y-%m"))
                            startDate = nrmlsDate.earliest_date
                            endDate = ApproxDate.from_iso8601(parser.parse(timeExpression_str.split()[1]+" "+timeExpression_str.split()[2]).strftime("%Y-%m")).latest_date
                        else:
                            nrmlsDate = ApproxDate.from_iso8601(str(sentence_row.time_expression))
                            startDate = nrmlsDate.earliest_date
                            endDate = nrmlsDate.latest_date
                        
                    if sentence_row.time_expression_type == "time_range":
                        if sentence_row.time_token == "YEAR_REGEX_1 LINKAGE_REGEX YEAR_REGEX_1" and (("to") in sentence_row.time_expression.lower() or
                                                                                                    ("and") in sentence_row.time_expression.lower()):
                            
                            startDate = ApproxDate.from_iso8601(parser.parse(sentence_row.time_expression.split()[0]).strftime("%Y")).earliest_date
                            endDate = ApproxDate.from_iso8601(parser.parse(sentence_row.time_expression.split()[2]).strftime("%Y")).latest_date
                            # print("value split ->",startDate,endDate)
                        else:
                            nrmlsDate = ApproxDate.from_iso8601(sentence_row.time_expression)
                            startDate = nrmlsDate.earliest_date
                            endDate = nrmlsDate.latest_date
                    
                    # print(nrmlsDate,refDate.earliest_date,refDate.latest_date)
                    # list_time_nrml.append([startDate,endDate])
                    # time_expression.append()
                except ValueError:
                    # print(ValueError)
                    # print(sentence_row.time_expression)
                    continue
                finally:
                    # print(nrmlsDate,refDate.earliest_date,refDate.latest_date)
                    list_time_nrml.append([startDate,endDate])
            timeExp_df = pd.DataFrame(list_time_nrml, columns=["normalised_value_start",'normalised_value_end'])
            
            try:
                biography_df = biography_df.drop(columns=["normalised_value_start",'normalised_value_end'], axis=1)
            except Exception as e: 
                print("ERROR: ", file_name_item)
                print(e)
                pass
            finally:
                biography_df = pd.concat([biography_df,timeExp_df],axis=1)
                
            biography_df.to_csv('extractedTimeExpressions/'+file_name_item.file_name,index=False)

2541302.csv
185202.csv
1310858.csv
297375.csv
300931.csv
167790.csv
185085.csv
760333.csv
699151.csv
15766.csv
371868.csv
767288.csv
1453044.csv
905775.csv
2345798.csv
168565.csv
175068.csv
155107.csv
243651.csv
529813.csv
1471350.csv
1150533.csv
226445.csv
50016.csv
52470803.csv
1111730.csv
973172.csv
211626.csv
460378.csv
1107893.csv
1372632.csv
141330.csv
3529094.csv
1084179.csv
1043762.csv
817466.csv
702942.csv
1417421.csv
3516152.csv
1010510.csv
709620.csv
192739.csv
1850886.csv
2488161.csv
423820.csv
777623.csv
467101.csv
87603.csv
2516568.csv
1291198.csv
264312.csv
214233.csv
833109.csv
97205.csv
735176.csv
143160.csv
3217228.csv
1505727.csv
879937.csv
1392224.csv
1341923.csv
3450589.csv
9700.csv
392501.csv
52625029.csv
1097923.csv
1194878.csv
171050.csv
2785324.csv
322567.csv
420376.csv
50876284.csv
1285387.csv
205720.csv
1244554.csv
1354841.csv
354680.csv
164576.csv
3180265.csv
298563.csv
2317280.csv
2738707.csv
2974145.csv
1714891.csv
2952161.csv
997083.csv
328940.csv
2519838

## USING LLMs to complete tempora expressions normalisation

Normalise all sentences that have a temporal expression

Filter only the HM and HT annotations

In [34]:
# LLM configuration using OpenAI
from unidecode import unidecode
import json
import time
from openai import OpenAI
import tiktoken
# Zero TEMPERATURE value to obtain the best answer
DEFAULT_TEMPERATURE = 0
DEFAULT_MAX_TOKENS = 500
DEFAULT_MODEL = "gpt-3.5-turbo"

In [35]:
# GETOPENAIKEY
def getOpenaiKey():
    return "OPENAI-KEY"

# openai.api_key = getOpenaiKey()
openai = OpenAI(
  api_key=getOpenaiKey()
)

In [36]:
# send prompt using API
def send_prompt(messages):
    resp = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=DEFAULT_TEMPERATURE,
        max_tokens=DEFAULT_MAX_TOKENS,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    # answer = resp.choices[0].message["content"]
    answer = resp.choices[0].message.content
    # print(resp.usage)
    print(answer)
    print(dict(resp).get('usage'))
    return answer

def send_prompt_name(target_text, sentence, biographyName):
    if len(biographyName)>0:
        system_msg = """You are a tool that extracts time references and returns results in ISO8601 date format. I will provide a sentence and the target text. 1. You should decide if the target text represents a time reference. 2. If the target text is not a specific time reference return "NO". 3. If the target text is a specific time reference then estimate the approximate dates in the context of the sentence. 4. If the information is not enough to calculate a date then use {}'s biography information.  5. Check the results are in ISO8601 format. 6. Return the results in JSON format using two keys: start_date and end_date. Estimate dates after point 4. Don't return fake dates.""".format(biographyName)
    else:
        system_msg = """You are a tool that extracts time references and returns results in ISO8601 date format. I will provide a sentence and the target text. 1. You should decide if the target text represents a time reference. 2. If the target text is not a specific time reference return "NO". 3. If the target text is a specific time reference then estimate the approximate dates in the context of the sentence. 4. Check the results are in ISO8601 format. 5. Return the results in JSON format using two keys: start_date and end_date. Estimate dates after point 4. Don't return fake dates."""
    messages = [
        {"role": "system", "content": system_msg},
        # {"role": "user", "content": f"Sentence: {message}"},
        {"role": "user", "content": f"Sentence: {sentence} Target text: {target_text}" },
    ]
    # print(messages)
    return send_prompt(messages)

def send_prompt_ranking(message, sentence):
    system_msg = """You are a tool that extracts time references and parses text to ISO 8601 date format. \
    Return the answer in json format using the following keys: 'start-date' and 'end-date'. \ 
    When is not possible to parse return empty values. \
    Use as context the following sentence: <{}>
            """.format(sentence)
    messages = [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": f"Sentence: {message}"},
    ]
    
    return send_prompt(messages)

In [37]:
def writeJSONFile(data,fileName):
    # You can also store the data in a file, for example, as a JSON file
    with open(fileName, "w") as json_file:
        json.dump(data, json_file)

def readJSONFile(fileName):
    # To read the stored JSON data back into Python:
    with open(fileName, "r") as json_file:
        stored_data = json.load(json_file)
        
    return stored_data

def getName(idBio):
    bios_df = pd.read_csv('TOTAL_download_biographies.csv')
    bios_df = bios_df[bios_df['id'] == int(idBio)]
    if len(bios_df)==0:
        return ''
    name = bios_df.iloc[0]['s']
    print(name)
    indexN = name.rfind('/')
    name = name[indexN+1:]#name.partition('/')[2]
    name = name.replace('_',' ')
    indexP = name.rfind('(')
    if indexP>0:
        name = name[:indexP]
    return name

In [31]:
# USING LLMs to complete time_expressions parsing
# Experiment 4 - BEST PERFORMANCE
# for chunk in pd.read_csv('../totalBiographiesBenchmark.csv', chunksize=50):
for chunk in pd.read_csv('toAnnotatePart6.csv', chunksize=3000):
    df_file_name = pd.DataFrame()
    df_file_name['file_name'] = chunk['file_name']
    # Control de number of tokens and transactions
    count = 0
    total_num_tokens = 0
    num_tokens = 0
    for file_name_item in df_file_name.itertuples():
        print("File: ",file_name_item.file_name)
        if os.path.isfile('cacheTimeExpressionsReferences/'+file_name_item.file_name.replace('.csv','.json')):
            print("Already in folder")
            continue
        # Obtain biography name
        bioName = getName(file_name_item.file_name.replace('.csv',''))
        # READ annotations
        meetupsAnnotations_df = pd.read_csv('meetupsAnnotations/'+file_name_item.file_name,usecols=['before','after','paragraphIndex','annotation'])
        # Filter only the HM and HT annotations
        meetupsAnnotations_df = meetupsAnnotations_df[~meetupsAnnotations_df['annotation'].isin(['N'])]
        
        if len(meetupsAnnotations_df) == 0:
            print("No annotations")
            continue
        
        if not os.path.isfile('extractedTimeExpressions/'+file_name_item.file_name):
            print("No time expressions")
            continue

        try:
            # 1. Read extracted Time expressions 2. drop any duplicates using ['time_expression','sentenceIndex','paragraphIndex'] 3. Filter only the NA values
            te_df = pd.read_csv('extractedTimeExpressions/'+file_name_item.file_name, usecols=['time_expression','sentenceIndex','paragraphIndex','normalised_value_start','normalised_value_end'])
            te_df = te_df.drop_duplicates(subset=['time_expression','sentenceIndex','paragraphIndex'])
        except pd.errors.EmptyDataError:    
            print("EmptyDataError: Time expression. No columns to parse from file")
            # What happens when there is no TIME? 
            continue
        except FileNotFoundError:
            print("FileNotFoundError: No time.")
            continue
        # Read the sentences
        sentences_df = pd.read_csv('indexedSentences/'+file_name_item.file_name,usecols=['sentences','sentenceIndex','paragraphIndex'])
        # Control errors
        errors_list = []
        list_time_nrml = []
        results_list = []
        
        # Execute two merge: a) for 'after' index and b) for 'before' index
        for index in ['after','before']:
            # print("INDEX", index)
            meetups_df = meetupsAnnotations_df[~meetupsAnnotations_df[index].isna()]
            meetups_df.rename(columns={index: 'sentenceIndex'}, inplace=True)

            # Filter all the TimeExpressions in MEETUPS ANNOTATIONS that are in index 'after' and 'before'
            te_index_df = pd.merge(meetups_df,te_df,how='inner', on=['sentenceIndex','paragraphIndex'])
            # te_index_df = te_df[te_df['paragraphIndex'].isin(meetups_df['paragraphIndex'])].copy()
            # te_index_df = te_df[te_df['sentenceIndex'].isin(meetups_df[index])].copy()
            te_index_df.sort_values(by=['normalised_value_start'],ascending=True,inplace=True)
            # print(te_index_df)
            te_index_df = te_index_df.drop_duplicates(subset=['sentenceIndex','paragraphIndex'])
            # print(te_index_df)
            if len(te_index_df) == 0:
                continue
                
            after_merged_df = pd.merge(sentences_df, te_index_df, how='inner', on=['sentenceIndex','paragraphIndex'])
            # print("after_merged_df", after_merged_df)
            encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
            print("Sentences to process: ", len(after_merged_df))
            for sentence_row in after_merged_df.itertuples():
                if sentence_row.normalised_value_start is not np.nan:
                    print("NA skip")
                    continue
                print("sentence_row: sent: ",str(sentence_row.sentenceIndex), " parag: ",str(sentence_row.paragraphIndex))
                time_expression_string = re.sub(r'([.?!])\1+', r'\1',re.sub(r'\.{3,}', '...',unidecode(str(sentence_row.time_expression).strip())))
                sentence_string = re.sub(r'([.?!])\1+', r'\1',re.sub(r'\.{3,}', '...',unidecode(str(sentence_row.sentences).strip())))

                try:
                    # print("START",sentence_row.time_expression)
                    print("1. Try send prompt")
                    answer = send_prompt_name(time_expression_string, sentence_string, bioName)
                    print("2. Response prompt")
                    # Parse the JSON response into a Python dictionary
                    data = json.loads(answer)
                    print(data)
                    # Count number of tokens
                    num_tokens = len(encoding.encode(answer))+200
                    
                    # if len(data) >0:
                    data["sentenceIndex"] = sentence_row.sentenceIndex
                    data["paragraphIndex"] = sentence_row.paragraphIndex
                    data["time_expression"] = sentence_row.time_expression
                    # data["sentences"] = sentence_string
                    results_list.append(data)
                    # print("results_list",results_list)
                except json.decoder.JSONDecodeError:
                    count +=1
                    total_num_tokens +=num_tokens
                    continue
                except Exception as e:
                    print("Identifier: ",str(sentence_row.sentenceIndex)," ",str(sentence_row.paragraphIndex))
                    traceback.print_exc()
                    print(e, file=sys.stdout)
                    # save errors in list
                    errors_list.append({"sentenceIndex":str(sentence_row.sentenceIndex),"paragraphIndex":str(sentence_row.paragraphIndex),
                                       "sentences":sentence_string,"time_expression": str(sentence_row.time_expression)})
                    print("waiting...")
                    time.sleep(30)
                    # raise
                finally:
                    count +=1
                    total_num_tokens +=num_tokens
                    print("total_num_tokens: ", total_num_tokens)
                    if total_num_tokens > 2500:
                        time.sleep(20)
                        total_num_tokens = 0
                    if (count >=4000):
                        print("Waiting on count...")
                        time.sleep(60)
                        count = 0
                    
        # check for response errors and run LLM read again
        errors_df = pd.json_normalize(errors_list)
        if len(errors_df) > 0:
            print("Try execute sentences with errors")
            for row in errors_df.itertuples():
                try:
                    print("1. Try send prompt")
                    time_expression_string = re.sub(r'([.?!])\1+', r'\1',re.sub(r'\.{3,}', '...',unidecode(str(row.time_expression).strip())))
                    sentence_string = re.sub(r'([.?!])\1+', r'\1',re.sub(r'\.{3,}', '...',unidecode(str(row.sentences).strip())))
                    answer = send_prompt_name(time_expression_string,sentence_string, bioName)
                    print("2. Response prompt")
                    # Parse the JSON response into a Python dictionary
                    data = json.loads(answer)
                    num_tokens = len(encoding.encode(answer))+200
                    print(data)
                    # if len(data) >0:
                    data["sentenceIndex"] = row.sentenceIndex
                    data["paragraphIndex"] = row.paragraphIndex
                    data["time_expression"] = row.time_expression
                    # data["sentences"] = sentence_string
                    results_list.append(data)
                except json.decoder.JSONDecodeError:
                    count +=1
                    total_num_tokens +=num_tokens
                    continue
                except Exception as e:
                    print("Identifier: "+str(row.sentenceIndex)+" "+str(row.paragraphIndex))
                    print(e, file=sys.stdout)
                    time.sleep(30)
                    # raise
                finally:
                    count +=1
                    total_num_tokens +=num_tokens
                    print("total_num_tokens: ", total_num_tokens)
                    if total_num_tokens > 2500:
                        time.sleep(20)
                        total_num_tokens = 0
                    if (count >=4000):
                        print("waiting...")
                        time.sleep(60)
                        count = 0
        # save final 
        writeJSONFile(results_list,'cacheTimeExpressionsReferences/'+file_name_item.file_name.replace('.csv','.json'))
        print("Waiting at file level...")
        time.sleep(2)
    print("waiting at batch level...")
    time.sleep(40)

File:  1965784.csv
Already in folder
File:  32264675.csv
Already in folder
File:  2934315.csv
Already in folder
File:  28312254.csv
Already in folder
File:  8040240.csv
Already in folder
File:  22459257.csv
Already in folder
File:  3362986.csv
Already in folder
File:  19474675.csv
Already in folder
File:  51596657.csv
Already in folder
File:  484364.csv
Already in folder
File:  4241227.csv
Already in folder
File:  46385848.csv
Already in folder
File:  52279001.csv
Already in folder
File:  8064528.csv
Already in folder
File:  5143989.csv
Already in folder
File:  15182641.csv
Already in folder
File:  6975970.csv
Already in folder
File:  1949333.csv
Already in folder
File:  2470630.csv
Already in folder
File:  18713679.csv
Already in folder
File:  23394028.csv
Already in folder
File:  1862380.csv
Already in folder
File:  14565496.csv
Already in folder
File:  21438871.csv
Already in folder
File:  43618783.csv
Already in folder
File:  37029067.csv
Already in folder
File:  8623467.csv
Alread

# FORMAT TIME EXPRESSIONS FOR KG CONSTRUCTION

In [38]:
# process LLM response cache
# reading every JSON file in cache
# return a list object of files in the given folder
for chunk in pd.read_csv('toAnnotatePart6.csv', chunksize=50):
    df_file_name = pd.DataFrame()
    df_file_name['file_name'] = chunk['file_name']
    # for each file
    for file_name_item in df_file_name.itertuples():
        print(file_name_item.file_name)

        if os.path.isfile('meetupsTimeExpressions/'+file_name_item.file_name):
            print("File already processed. Next")
            continue
        copy_file = False  
        try:
            dataJSON = readJSONFile('cacheTimeExpressionsReferences/'+file_name_item.file_name.replace('.csv','.json'))
            llm_df = pd.json_normalize(dataJSON)
            # print(dataJSON)
            llm_df['source'] = 'LLM'
            # print(llm_df)
            if len(llm_df)==0:
                print("Empty File. No results Time extraction with LLM.")
                copy_file = True
            else:
                # Filter rows that contain 'NO' in start date or end_date
                llm_df = llm_df[~((llm_df['start_date'].isin(['NO']))|(llm_df['end_date'].isin(['NO'])))]
                # Validate expected format. # Define the expected date format using a regular expression
                expected_format = r'\d{4}-\d{2}-\d{2}'
                # Filter rows where the 'DateColumn' does not match the expected format
                llm_df = llm_df[llm_df['start_date'].str.match(expected_format, na=False)]
                llm_df = llm_df[llm_df['end_date'].str.match(expected_format, na=False)]
            # print(llm_df)
            # "start_date", "end_date", "sentenceIndex", "paragraphIndex", "time_expression"
        except pd.errors.EmptyDataError:    
            print("EmptyDataError: No results Time extraction with LLM.")
            copy_file = True
            # continue
        except FileNotFoundError:
            print("FileNotFoundError: No time extraction file.")
            copy_file = True
            # continue
        except Exception as e:
            print("Identifier: ",file_name_item.file_name)
            traceback.print_exc()
            print(e, file=sys.stdout)

        new_te_df = pd.DataFrame()
        try:
            # 1. Read extracted Time expressions 2. drop any duplicates using ['time_expression','sentenceIndex','paragraphIndex'] 3. Filter only the NA values
            te_df = pd.read_csv('extractedTimeExpressions/'+file_name_item.file_name) #, usecols=['time_expression','sentenceIndex','paragraphIndex','normalised_value_start','normalised_value_end'])
            # print("read",len(te_df))
            te_df = te_df.drop_duplicates(subset=['time_expression','sentenceIndex','paragraphIndex','normalised_value_start','normalised_value_end'])
            # print("drop",len(te_df))
            if copy_file:
                print("No LLM results, copy file anyway.")
                te_df['source'] = 'ML'
                te_df.to_csv('meetupsTimeExpressions/'+file_name_item.file_name.replace(".json",".csv"), index=False)
                continue
            # No NA values in columns start date
            te_nna_df = te_df[~te_df['normalised_value_start'].isna()]
            # NA values in column start date
            te_na_df = te_df[te_df['normalised_value_start'].isna()]
            
            na_merge = pd.merge(llm_df,te_na_df,how='inner', on=['sentenceIndex','paragraphIndex','time_expression'])
            # print(na_merge)
            # Remove two columns: 'normalised_value_start','normalised_value_end'
            na_merge.drop(['normalised_value_start','normalised_value_end'], axis=1, inplace=True)
            # Rename columns 
            na_merge.rename(columns={'start_date': 'normalised_value_start','end_date':'normalised_value_end'}, inplace=True)
            na_merge = na_merge[~na_merge['normalised_value_start'].isna()]
            
            # concat 
            te_na_df['source'] = 'ML'
            na_merge = pd.concat([te_na_df,na_merge],ignore_index = True)
            na_merge.sort_values(by=['normalised_value_start'],ascending=False,inplace=True)
            # print("read",len(na_merge))
            na_merge.drop_duplicates(subset=['paragraphIndex','sentenceIndex'],inplace=True)
            # print("drop",len(na_merge))
            
            # te_df['source'] = 'ML'
            # new_te_df = pd.concat([te_df,na_merge],ignore_index = True)
            te_nna_df['source'] = 'ML'
            new_te_df = pd.concat([te_nna_df,na_merge],ignore_index = True)
            # Sort and Drop duplicates
            # new_te_df.sort_values(by=['normalised_value_start','source'],ascending=False,inplace=True)
            # print("read",len(new_te_df))
            # new_te_df.drop_duplicates(subset=['paragraphIndex','sentenceIndex'],inplace=True)
            # print("drop",len(new_te_df))
            new_te_df.to_csv('meetupsTimeExpressions/'+file_name_item.file_name.replace(".json",".csv"), index=False)
        except pd.errors.EmptyDataError:    
            print("EmptyDataError: No time extraction with libraries.")
            continue 
        except Exception as e:
            print("Identifier: ",str(row['sentenceIndex'])," ",str(row['paragraphIndex']))
            traceback.print_exc()
            print(e, file=sys.stdout)

1965784.csv


/tmp/ipykernel_767405/4058922566.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  te_na_df['source'] = 'ML'
/tmp/ipykernel_767405/4058922566.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  te_nna_df['source'] = 'ML'
/tmp/ipykernel_767405/4058922566.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

32264675.csv
2934315.csv
28312254.csv
8040240.csv
Empty File. No results Time extraction with LLM.
No LLM results, copy file anyway.
22459257.csv
3362986.csv


/tmp/ipykernel_767405/4058922566.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  te_na_df['source'] = 'ML'
/tmp/ipykernel_767405/4058922566.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  te_nna_df['source'] = 'ML'
/tmp/ipykernel_767405/4058922566.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

19474675.csv
Empty File. No results Time extraction with LLM.
No LLM results, copy file anyway.
51596657.csv
484364.csv
4241227.csv
46385848.csv
52279001.csv


...

/tmp/ipykernel_767405/4058922566.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  te_na_df['source'] = 'ML'
/tmp/ipykernel_767405/4058922566.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  te_nna_df['source'] = 'ML'
/tmp/ipykernel_767405/4058922566.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

3429561.csv
43618191.csv
104987.csv
32702085.csv
36819211.csv
12706336.csv
2349481.csv
16553122.csv
8569551.csv
13520712.csv
2084339.csv


/tmp/ipykernel_767405/4058922566.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  te_na_df['source'] = 'ML'
/tmp/ipykernel_767405/4058922566.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  te_nna_df['source'] = 'ML'
/tmp/ipykernel_767405/4058922566.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

51060074.csv
4222804.csv
7950876.csv
12949776.csv
42480529.csv
6141586.csv


/tmp/ipykernel_767405/4058922566.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  te_na_df['source'] = 'ML'
/tmp/ipykernel_767405/4058922566.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  te_nna_df['source'] = 'ML'
/tmp/ipykernel_767405/4058922566.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde